We load data from fashion MNIST, where personal items of clothing to stand in for personal images and train a naive Bayes classifier to distinguish two classes in the “fashion MNIST” dataset, t-shirts and sneakers.

`trainX`, `trainY`, `testX`, and `testY` are the full set of training and testing examples from the dataset, where `X` represents the instances and `Y` the labels.

In [ ]:
# Load fashion MNIST, split into training/testing
!pip install keras=='2.3.1'
from keras.datasets import fashion_mnist
((trainX, trainY), (testX, testY)) = fashion_mnist.load_data()

Using TensorFlow backend.


4423680/4422102 [==============================] - 1s 0us/step


We import the Gaussian naive Bayes model we'll be using for training as well as numpy for additional functions used throughout the notebook.

The instances are arranged as 28 by 28 grayscale images. For training the model, we `reshape` the data into flat vectors of size 784.

To simplify training, we convert the grayscale images to black and white. For each image, in the training and testing set, we compute the `median` grayscale value and call everything below that value zero and everything about that value one.

In [ ]:
# from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import numpy as np

# flatten data
trainX = np.reshape(trainX,(len(trainX),-1))
testX = np.reshape(testX,(len(testX),-1))

# grayscale to B&W
testX = testX > np.reshape(np.median(testX,1),(-1,1))
trainX = trainX > np.reshape(np.median(trainX,1),(-1,1))

The fashion MNIST dataset includes ten classes, but we’re onlyll just looking at using class 0, which are t-shirts, and class 7, which are sneakers. 

`testYpair`, `testXpair`, `trainYpair`, and `trainXpair` are subsets of the training and testing data where only the pair of target classes are retained.

The dataset includes 12000 training examples and 2000 testing examples, evenly split across the two categories.

In [ ]:
class0 = 0 # top
class1 = 7 # sneaker
testYpair = 0+np.compress(np.logical_or(testY == class0, testY == class1), testY)
testXpair = 0+np.compress(np.logical_or(testY == class0, testY == class1), testX, axis=0)
trainYpair = 0+np.compress(np.logical_or(trainY == class0, trainY == class1), trainY)
trainXpair = 0+np.compress(np.logical_or(trainY == class0, trainY == class1), trainX, axis=0)

Below we define the function `showpic` to display images in the data.

In [ ]:
import keras.preprocessing.image as img

def showpic(arr):
  onec = np.reshape(arr,(28,28,1))
  onecbw = 255*np.concatenate((onec,onec,onec),2)
  display(img.array_to_img(onecbw,scale=False))

Here we display 10 images from `trainXpair` with their corresponding `trainYpair` label.

In [ ]:
for i in range(10):
  print(trainYpair[i])
  showpic(trainXpair[i])

0


0


0


7


0


7


0


0


0


7


We then train the Gaussian naive Bayes model `gnb` and get its predicitons for the test set, `predYpair`.  

We then print the number of muislabeled points.

In [ ]:
gnb = GaussianNB()
predYpair = gnb.fit(trainXpair, trainYpair).predict(testXpair)
print("Number of mislabeled points out of a total %d points : %d"
       % (testXpair.shape[0], (testYpair != predYpair).sum()))

Number of mislabeled points out of a total 2000 points : 3


This code blocks shows how the predictions change when we randomly remove 1 example from the training data and train another model, `gnb2`, on that new training data `trainXpair2` and `trainYpair2`.  The number of mislabeled test datapoints when using `gnb2` are printed below.

In [ ]:
# remove one at random

remove = np.random.randint(0,len(trainXpair)+1)
removedX = trainXpair[remove]
removedY = trainYpair[remove]
trainXpair2 = np.concatenate((trainXpair[:remove],trainXpair[(remove+1):]))
trainYpair2 = np.concatenate((trainYpair[:remove],trainYpair[(remove+1):]))
gnb2 = GaussianNB()
predYpair2 = gnb2.fit(trainXpair2, trainYpair2).predict(testXpair)
print("Number of mislabeled points out of a total %d points : %d"
       % (testXpair.shape[0], (testYpair != predYpair2).sum()))

Number of mislabeled points out of a total 2000 points : 3


Below we take the average of the images from each class `trainXpair2`. `example0` is the average of data belonging of the shoe class.  The average of the data belonging to the shirt class is `example1`.

In [ ]:
example0 = np.mean(np.compress(trainYpair2 == class0, trainXpair2, axis=0),axis=0)
example1 = np.mean(np.compress(trainYpair2 == class1, trainXpair2, axis=0),axis=0)

This `binarysearch` function simulates a `query` image to estimate the image we have removed from the dataset used in training. It sumulates `query` by learning a weighted average of the images in `example1` and `example0` so that the `model` classifies the `query` as roughly 50 percent t-shirts and 50 percent sneakers.  The weighting parameter, `mid`, is updated based on the predictions of the `model`.  If the model predicts a probability is less than 0.5, we update `right` so that `mid` decreases, otherwise we updates `left` so that `mid` increases.  When `right` and `left` are close in value, we return our weighted image.

In [ ]:
# binary search

def binarysearch(model):
  left = 0
  right = 1

  while (right-left > 0.00000001):
    mid = (right+left)/2
    exampleMid = np.reshape((1-mid)*example0+mid*example1,(1,-1))
    if model.predict_proba(exampleMid)[0][0] < 0.5:
      right = mid
      
    else:
      left = mid
  query = np.reshape((1-mid)*example0+mid*example1,(1,-1))
  return(query)

Our `attack` on original model learns a reconstruction of the removed image `recon`.  Starting with `query` and synthetic image that the trained model classifies as roughly 50 percent t-shirts,we see how each of the two models classifies the test instance, the original model and model trained without the target data item.

Then, for each column, we replace all values in a column with 1. We again see how the original model and the new model to classify this instance and note which one reacts more “strongly” to the change by changing its class prediction.

The reconstruction of the image is exactly the pattern of which components cause more of a reaction in the original model or the retrained model.

We display the reconstruction, `recon`, below.

In [ ]:
# Let's say we know the label of the removed data point.
# Wouldn't it be the case that moving from vector v to v_{i=val} would make THAT
# label more likely i...

# We train up two different NB models. Assume they differ in class0 by a single 
# vector m.
# The models for class1 should be the same.
# Let's consider two inputs x and x' that differ only in dimension i.
# The old model should be M = sum_{v in class0} v / # instances in class0.
# The new model should be M' = (sum_{v in class0} v - m) / # instances in class0-1
# Roughly, we are looking at M.x, M'.x, M.x', M'.x'.

# M'.x ~ (M-v).x
# => M.x - v.x
# So, M.x - M'.x ~ v.x
# So, (M.x - M'.x) - (M.x' - M'.x') ~ v_i


# hmmm. So, if example0[i] and example1[i] are the same, might as well choose 
# that value. We won't see much evidence that will separate the two cases 
# anyway.
# If example0[i] is bigger than example1[i], then putting a 1 at position i
# should make class 0 more likely. If the likelihood EHNANCEMENT is positive,
# then the missing vector probably has a 1 there, too.

def attack(gnb, gnb2, example0, example1):
  recon = []
  query = binarysearch(gnb)
  z = np.reshape(query,(1,-1))
  z2 = np.reshape(query,(1,-1))
  for i in range(len(trainXpair[0])):
    e = np.copy(z)
    e[0][i] = 1.0
    pz = gnb.predict_proba(z)[0][1]
    pe_1 = gnb.predict_proba(e)[0][1]
    strength = pe_1*(pz-1) / (2 *pe_1*pz - pe_1 - pz)

    e[0][i] = 0.0
    pe_0 = gnb.predict_proba(e)[0][1]

    e2 = np.copy(z2)
    e2[0][i] = 1.0
    pz2 = gnb2.predict_proba(z2)[0][1]
    pe2_1 = gnb2.predict_proba(e2)[0][1]
    e2[0][i] = 0.0
    pe2_0 = gnb2.predict_proba(e2)[0][1]
    strength2 = pe2_1*(pz2-1) / (2 *pe2_1*pz2 - pe2_1 - pz2)

    e2[0][i] = 0.0
    pe2_0 = gnb2.predict_proba(e)[0][1]

    if abs(example0[i] - example1[i]) < 0.001: recon += [example0[i]]
    else:
      if strength > strength2:
        recon += [0]
      else:
        recon += [1]
  return(recon)

recon = attack(gnb, gnb2, example0, example1)

showpic(recon)

(1, 784)
(1, 784)
(1, 784)
(1, 784)
(1, 784)
(1, 784)
(1, 784)
(1, 784)
(1, 784)
(1, 784)
(1, 784)
(1, 784)
(1, 784)
(1, 784)
(1, 784)
(1, 784)
(1, 784)
(1, 784)
(1, 784)
(1, 784)
(1, 784)
(1, 784)
(1, 784)
(1, 784)
(1, 784)
(1, 784)
(1, 784)


Below we show the image that was removed.

In [ ]:
showpic(removedX)

Below is the image that we learned using `attack`.

In [ ]:
showpic(1-np.array(recon))

Now we'll use IBM's `diffprivlib` to train differentially private models.

In [ ]:
!pip install diffprivlib

fatal: destination path 'differential-privacy-library' already exists and is not an empty directory.


Below we import the differentially private version of Gaussian naive Bayes and train the model, `gnb`, on our original data.  We print the accuracy of the model below.

In [ ]:
import diffprivlib.models as dpmodel

gnb = dpmodel.GaussianNB()
predYpair = gnb.fit(trainXpair, trainYpair).predict(testXpair)
print("Number of mislabeled points out of a total %d points : %d"
       % (testXpair.shape[0], (testYpair != predYpair).sum()))

/usr/local/lib/python3.6/dist-packages/diffprivlib/models/naive_bayes.py:93: PrivacyLeakWarning: Bounds have not been specified and will be calculated on the data provided. This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify bounds for each dimension.
  "privacy leakage, specify bounds for each dimension.", PrivacyLeakWarning)


Number of mislabeled points out of a total 2000 points : 57


Again we remove a single datapoint from the training set and train another differentially private naive Bayes model.  We report its performance below.

In [ ]:
# remove one

# remove = np.random.randint(0,len(trainXpair)+1)
removedX = trainXpair[remove]
removedY = trainYpair[remove]
trainXpair2 = np.concatenate((trainXpair[:remove],trainXpair[(remove+1):]))
trainYpair2 = np.concatenate((trainYpair[:remove],trainYpair[(remove+1):]))
gnb2 = dpmodel.GaussianNB()
predYpair2 = gnb2.fit(trainXpair2, trainYpair2).predict(testXpair)
print("Number of mislabeled points out of a total %d points : %d"
       % (testXpair.shape[0], (testYpair != predYpair2).sum()))

/usr/local/lib/python3.6/dist-packages/diffprivlib/models/naive_bayes.py:93: PrivacyLeakWarning: Bounds have not been specified and will be calculated on the data provided. This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify bounds for each dimension.
  "privacy leakage, specify bounds for each dimension.", PrivacyLeakWarning)


Number of mislabeled points out of a total 2000 points : 54


Next we try to rerun the `attack` we had showed earlier and display the `recon` image we have learned.

In [ ]:
example0 = np.mean(np.compress(trainYpair2 == class0, trainXpair2, axis=0),axis=0)
example1 = np.mean(np.compress(trainYpair2 == class1, trainXpair2, axis=0),axis=0)

recon = attack(gnb, gnb2, example0, example1)

showpic(recon)